### Build CNN

In [1]:
from keras.models import Sequential
from keras.layers import (Convolution2D, MaxPooling2D, Flatten, Dense)

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
# Initialize CNN
classifier = Sequential()

In [4]:
# Add Convolutional layer
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

In [5]:
# Add Pooling layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [6]:
# To improve performance
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Convolution2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [7]:
# Add Flatten layer
classifier.add(Flatten())

In [8]:
# Add hidden layer (full connection)
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

In [9]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Fitting the CNN using images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000/32,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000/32)

Epoch 1/5
250/250 [==============================] - 220s 878ms/step - loss: 0.6619 - acc: 0.5909 - val_loss: 0.6112 - val_acc: 0.6470
Epoch 2/5
250/250 [==============================] - 212s 847ms/step - loss: 0.5978 - acc: 0.6765 - val_loss: 0.5542 - val_acc: 0.7185
Epoch 3/5
250/250 [==============================] - 210s 840ms/step - loss: 0.5485 - acc: 0.7200 - val_loss: 0.6001 - val_acc: 0.6765
Epoch 4/5
250/250 [==============================] - 241s 966ms/step - loss: 0.5114 - acc: 0.7450 - val_loss: 0.4922 - val_acc: 0.7640
Epoch 5/5
250/250 [==============================] - 249s 996ms/step - loss: 0.4880 - acc: 0.7634 - val_loss: 0.4569 - val_acc: 0.7765


In [15]:
from keras.preprocessing import image
test_image = image.load_img('cat.8.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [16]:
result

array([[  9.11639290e-05]], dtype=float32)

In [15]:
# serialize model to JSON
model_json = classifier.to_json()
with open("classifier.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
# load json and create model
json_file = open('classifier.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
from skimage.io import imread
from skimage.transform import resize
 
class_labels = {v: k for k, v in training_set.class_indices.items()}
 
img = imread('IMG_0399.jpg') #make sure that path_to_file contains the path to the image you want to predict on. 
img = resize(img,(64,64))
img = np.expand_dims(img,axis=0)
 
if(np.max(img)>1):
    img = img/255.0
 
prediction = classifier.predict_classes(img)
print(training_set.class_indices)
print(prediction[0][0])

{'cats': 0, 'dogs': 1}
0


/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
